In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

Notebook to sketch out implementation of QAOA algorithm in pytorch.

Final impementation in `k_sat/pytorch_solver/`

In [2]:
import torch
import numpy as np

In [3]:
from math import log
def t_to_sv(t):
  t = t.detach().clone()
  # For easier visualisation
  t.real[torch.abs(t.real) < 1e-6] = 0
  t.imag[torch.abs(t.imag) < 1e-6] = 0
  return Statevector(t.numpy())


In [4]:
from qiskit.quantum_info.states.statevector import Statevector 
real = torch.tensor([1., 0.])
imag = torch.tensor([0., 0.])
a = torch.complex(real, imag)
t_to_sv(a).draw("latex")

<IPython.core.display.Latex object>

Application of cost unitary:

In [5]:
def cost(z, h, g_j):
  hg = torch.complex(torch.tensor(0.), h * g_j)
  hg_exp = torch.exp(hg)
  z = hg_exp * z
  return z

Application of mixing unitary:

In [6]:
def mix(z, g_j):
  cg = torch.complex(torch.cos(g_j), torch.tensor(0.))
  sg = torch.complex(torch.tensor(0.), torch.sin(g_j))

  for i in range(n):
    cz = cg * z

    # swap indices
    z = z.reshape((2,) * n)
    z = z.transpose(0, i)
    fh, sh = z.split(1)
    z = torch.cat((sh, fh))
    z = z.transpose(0, i)
    z = z.reshape(N)

    # reduce for iteration to continue
    z = cz + sg * z
  
  return z



E.g. we'd expect $\ket{000} + \ket{001}$ to be mixed to $\ket{111} + \ket{100}$ when using an angle $\beta_j = \frac{\pi}{2}$

In [7]:
n = 3
N = 2 ** n
real = torch.zeros(size=(N,), dtype=torch.float32)
imag = torch.zeros(size=(N,), dtype=torch.float32)

# |000> + |001> 
real[0] = 1
real[1] = 1
y = torch.complex(real, imag)

ry = mix(y, torch.tensor(torch.pi/2))

In [8]:
t_to_sv(y).draw("latex")

<IPython.core.display.Latex object>

In [9]:
t_to_sv(ry).draw("latex")

<IPython.core.display.Latex object>

Putting this all together:

In [10]:
def qaoa(z, gamma, beta, p, h):

  # Apply layers
  for j in range(p):
    z = cost(z, h, gamma[j])
    z = mix(z, beta[j])
  
  return z

def succ_prob(circuit, hS):
  circuit = (circuit * circuit.conj()).real
  return torch.dot(circuit, hS)

In [11]:
from formula.cnf import CNF
from formula.clause import Clause
from formula.variable import Variable
x0 = Variable(0, False) 
x1 = Variable(1, False) 
x2 = Variable(2, False) 
notx0 = Variable(0, True) 
notx1 = Variable(1, True) 
notx2 = Variable(2, True) 

c1 = Clause([x0, x1, x2])
c2 = Clause([x0, x1, notx2])
c3 = Clause([x0, notx1, x2])
c4 = Clause([x0, notx1, notx2])
c5 = Clause([notx0, x1, x2])
c6 = Clause([notx0, x1, notx2])
c7 = Clause([notx0, notx1, x2])

cnf = CNF([c1, c2, c3, c4, c5, c6, c7])

In [25]:
p = 3
n = 3
N = 2 ** n

# Starting state
coeff = 1 / (2 ** (n / 2))
s = torch.full((N, ), coeff, dtype=torch.cfloat)

# initial values
gamma_i = torch.full(size=(p, ), fill_value=-0.01)
beta_i = torch.full(size=(p, ), fill_value=0.01)

# optimisable parameters
gamma = torch.nn.Parameter(gamma_i)
beta = torch.nn.Parameter(beta_i)

# Optimisation
epochs = 1000
optimiser = torch.optim.Adam([gamma, beta], lr=0.01, maximize=True)

for i in range(epochs):
  optimiser.zero_grad()
  z = qaoa(s, gamma, beta, p, cnf.naive_counts)
  p_succ = succ_prob(z, cnf.naive_sats)
  p_succ.backward()
  optimiser.step()
  if i % 100 == 0:
    print(f'Success probability: {p_succ.item()}')

Success probability: 0.1254493147134781
Success probability: 0.7080141305923462
Success probability: 0.9788187742233276
Success probability: 0.9997613430023193
Success probability: 0.9999926090240479
Success probability: 0.9999998807907104
Success probability: 1.0
Success probability: 1.0
Success probability: 0.9999996423721313
Success probability: 0.9999995231628418


Check output state of circuit

In [26]:
gamma_f = gamma.detach().clone()
beta_f = beta.detach().clone()

output = qaoa(s, gamma_f, beta_f, p, cnf.naive_counts)

In [27]:
t_to_sv(output).draw("latex")

<IPython.core.display.Latex object>

In [28]:
t_to_sv(output).probabilities_dict(decimals=3)

{'111': 1.0}

In [24]:
#from torch.profiler import profile, ProfilerActivity
#
## Starting state
#coeff = 1 / (2 ** (n / 2))
#s = torch.full((N, ), coeff, dtype=torch.cfloat)
#init_gamma = torch.full(size=(1, ), fill_value=-0.01)
#init_beta = torch.full(size=(1, ), fill_value=0.01)
#gamma = torch.nn.Parameter(init_gamma)
#beta = torch.nn.Parameter(init_beta)
#out = qaoa(s, gamma, beta, 1, cnf.naive_counts)
#with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
#	out = qaoa(s, gamma, beta, 1, cnf.naive_counts)
#	succ_prob(out, cnf.naive_sats)
#print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))